### A notebook for data exploration and function testing for predict sales web app 

### Contents:
1. Data exploration
2. Feature engineering and train,test and predict split
3. Creating a base estimator to compare with
4. Creating and testing linear model
5. Creating and testing XGBoost model 
6. Testing on unseen shops
7. Conclusions

## 1. Data exploration

### A long cell with functions from app/process_data.py - you can ignore it and scroll past 

In [1]:
# import libraries
import sys
import pandas as pd
import datetime
import math
import numpy as np


def preprocess_data(sales_filepath, items_filepath, use_shop_ids=[38,42,35,23,32,24,4,5,12,29]):
    '''
        preprocess_data() - function that transforms initial training data CSV files into a dataframe 
                            in a format that can be then processed and used in training pipeline
        Input:
            sales_filepath - (str) path to training sales data 
            items_filepath - (str) path to training data on items (names, categories)
            use_shop_ids - (list) a subset of int shop ids from 'sales_filepath' CSV file to use during training to reduce training time
        Output:
            data - (pd.DataFrame) a Pandas dataframe with sales data

    '''
    sales_train = pd.read_csv(sales_filepath)
    items = pd.read_csv(items_filepath)
    data = sales_train[sales_train['shop_id'].isin(use_shop_ids) ].drop(columns=['date_block_num']).reset_index(drop=True)
    data = pd.merge(data, items, on=['item_id'], how='left')    

    return data

def check_data_correctnes(data):
    '''
        check_data_correctnes() - function that checks correctness of a dataframe format and data, and assigns a shop_id if none is given
                                  in case of a single shop in the data
        Input:
            data - (pd.DataFrame) a Pandas dataframe with sales data
            Needed columns are: date, shop_id, item_id, item_price, item_cnt_day, item_name, item_category_id
        Output:
            data - (pd.DataFrame) a Pandas dataframe with sales data with necessary changes if needed (add shop_id if empty),
            OR
            raises an exception to let the web app know that format is incorrect 
    '''
    data['shop_id'] = data['shop_id'].fillna('shop1')
    return data

def clean_and_aggreagate(data):
    '''
        clean_and_aggreagate() - function that cleanes, aggregates and sorts a Pandas dataframe for training or predicting sales
        Input:
            data - (pd.DataFrame) a Pandas dataframe with sales data
        Output:
            data_monthly - (pd.DataFrame) a cleaned and aggregated sorted dataframe
    '''

    # delete data with negative item counts or prices
    data = data[data['item_cnt_day']>0]
    data = data[data['item_price']>0]
    # clean from any null values
    data = data.dropna()
    
    # convert date from string to datetime format
    data['date'] = pd.to_datetime(data['date'], format='%d.%m.%Y')

    # Add month and year and aggregate data
    data['month'] = data['date'].dt.month
    data['year'] = data['date'].dt.year

    # making sure we are getting only needed columns
    keep_columns_agg = ['shop_id', 'item_id', 'item_price', 'item_cnt_day', 'month', 'year', 'item_category_id']
    
    # aggregate data monthly and by shops and items 
    data_monthly = data[keep_columns_agg].groupby(['year','month', 'shop_id', 'item_category_id', 'item_id'], as_index=False).agg(
        {'item_price':'mean', 
        'item_cnt_day':['sum', 'mean'] })

    # make multilevel index flat
    data_monthly.columns = data_monthly.columns.map(''.join)

    # rename columns
    data_monthly = data_monthly.rename(columns={
        'item_pricemean': 'item_price_avg', 
        'item_cnt_daysum':'item_cnt_month',
        'item_cnt_daymean':'item_cnt_day_mean'})
    
    # sort the dataframe for future analysis
    data_monthly = data_monthly.sort_values(by=['year', 'month', 'shop_id', 'item_id']).reset_index(drop=True)

    return data_monthly

def add_empty_rows(data_monthly):
    '''
        add_empty_rows() - function that extends a dataframe of aggregated sales with skipped rows
                           so that the dataframe has explicit data on zero sales (sets 'item_cnt_month' to zero) 
                           instead of not mentioning an item for a particular month in a particular shop
        Input:
            data_monthly - (pd.DataFrame) sorted Pandas dataframe with aggregated sales data
        Output:
            data_monthly_ext - (pd.DataFrame) extended dataframe with explicit 0 rows
    '''
    item_ids = data_monthly['item_id'].unique()
    shop_ids = data_monthly['shop_id'].unique()

    start_date_year = int(data_monthly.iloc[0,:]['year'])
    start_date_month = int(data_monthly.iloc[0,:]['month'])
    end_date_year = int(data_monthly.iloc[-1,:]['year'])
    end_date_month = int(data_monthly.iloc[-1,:]['month'])

    # --- calculate total number of months in the period of historical data
    number_of_months = (end_date_year - start_date_year) * 12 + (end_date_month - start_date_month) + 1

    # will have data on all possible combinations of sales records
    # for given shops, items and given time period 
    zero_rows = []
    cur_month = start_date_month
    cur_year = start_date_year

    for i in range(number_of_months):
        for shop in shop_ids:
            for item in item_ids:
                # adding a row - transaction for certain shop, year, month and item that will be zeroes initially
                # and then joined with main data so that main data has explicit zeros as transactions
                row = [cur_year, cur_month, shop, item]
                zero_rows.append(row)
        
        # create a correction coeeficient for the case of December, 12th month, so that we het 12 and not 0
        add_12_if_receive_0 = (12 - 12*math.ceil((cur_month+1)%12 / 12))
        # increase month by 1, make sure it will be in [1,12] range
        cur_month = (cur_month+1)%12 + add_12_if_receive_0
        
        # increase year if we arrived to the 1st month, January, after increasing month
        if cur_month == 1:
            cur_year += 1

    # create a df of zero rows    
    zero_rows = pd.DataFrame(zero_rows, columns=['year', 'month', 'shop_id', 'item_id'])

    # merge zero rows with the main data, data_monthly
    data_monthly_ext = pd.merge(zero_rows, data_monthly, on=['year','month', 'shop_id','item_id'], how='left')
    # fill missing records with 0s
    data_monthly_ext.fillna(0, inplace=True)
    # make sure the dataframe is sorted
    data_monthly_ext = data_monthly_ext.sort_values(by=['year', 'month', 'shop_id', 'item_id']).reset_index(drop=True)

    return data_monthly_ext

def add_global_features(data_monthly_ext):
    '''
        add_global_features() - function that adds features for training and predicting purposes.
                                I called features global becasue they can be applied to the whole data
                                before splitting into train and test without any data leakage in terms of forecasting
        Input:
            data_monthly_ext - (pd.DataFrame) a sorted by year and month Pandas dataframe with aggregated sales data
        Output:
            data_monthly_ext - (pd.DataFrame) initial dataframe with new features
    '''
    # --- add date_block to split data into train and test
    start_month = data_monthly_ext.iloc[0,:]['month']
    start_year = data_monthly_ext.iloc[0,:]['year']

    # order of the starting month - we will subtract it from other month to understand how many month we moved forward
    starting_month_agg = start_year*12 + start_month
    # create arrays of year and month to perform calculations on them using np.vectorize
    years = np.array(data_monthly_ext['year'])
    months = np.array(data_monthly_ext['month'])
    # for better performance create a vectorized function that computes block number (order of month from starting month)
    calculate_block_num = np.vectorize(lambda year, month: (year*12 + month)- starting_month_agg)
    # number of months since year 0 in starting date - this will be subtracted to understand how many months we have moved ahead from start
    data_monthly_ext['date_block_num'] = calculate_block_num(years, months)

    # Feature engineering
    # add rolling statistics

    # define size of a rolling window
    rolling_window_size = 3
    # --- Calculate rolling statistics
    # keep functions in a list to iterate
    functions = [lambda column: column.rolling(window=rolling_window_size, min_periods=1).mean(), 
                lambda column: column.rolling(window=rolling_window_size, min_periods=1).max(), 
                lambda column: column.rolling(window=rolling_window_size, min_periods=1).min(), 
                lambda column: column.rolling(window=rolling_window_size, min_periods=1).std()]
    # these are suffixes to add to column names to generate new names
    suffixes = ['mean', 'max', 'min', 'std']

    # create len(functions) new features
    for suff, func in zip(suffixes, functions):
        data_monthly_ext['item_cnt_roll_{}'.format(suff)] = data_monthly_ext.groupby(['shop_id','item_id'])['item_cnt_month'].apply(func)

    # Fill the empty std features with 0
    data_monthly_ext['item_cnt_roll_std'] = data_monthly_ext['item_cnt_roll_std'].fillna(0)

    # store average number of items sold per month for each item in a shop up to this point using pd.expanding function
    data_monthly_ext['item_mean_past'] = data_monthly_ext.groupby(['shop_id', 'item_category_id'])[['item_cnt_month']].expanding().mean().values

    # store average number of items sold per month for each category in a shop up to this point using pd.expanding function
    data_monthly_ext['category_mean_past'] = data_monthly_ext.groupby(['shop_id', 'item_id'])[['item_cnt_month']].expanding().mean().values


    # let us calculate lags for the past 3 months
    number_lags = range(1,4)

    # --- generate shifted number of items sold from the past 1-3 months
    for lag in number_lags:
        feature_name = 'item_cnt_shifted{}'.format(lag)
        data_monthly_ext[feature_name] = data_monthly_ext.sort_values('date_block_num').groupby(['shop_id', 'item_id'])['item_cnt_month'].shift(lag)
        # Fill the empty shifted features with 0
        data_monthly_ext[feature_name] = data_monthly_ext[feature_name].fillna(0)

    # generate trend which shows the change in item sales count
    # trend = current - (previous_1 + ... + previous_n)/n = n*current - previous_1 - ... - previous_n, in our case n=3
    # initially fill with current sales multiplied by number of times we will substract previous values to get average
    data_monthly_ext['item_trend'] = len(number_lags) * data_monthly_ext['item_cnt_month']
    # then subtract previous n values
    for lag in number_lags:
        feature_name = 'item_cnt_shifted{}'.format(lag)
        data_monthly_ext['item_trend'] -= data_monthly_ext[feature_name]
    # then divide by the number of times we have subtracted previous values to get average
    data_monthly_ext['item_trend'] /= len(number_lags) 

    return data_monthly_ext


def add_labels(data_monthly_ext):
    '''
        add_labels() - function that creates labels = a column of values we will predict = sales next month for an item in a shop
        Input:
            data_monthly_ext - (pd.DataFrame) a Pandas dataframe with aggregated sorted sales data
        Output:
            data_monthly_ext - (pd.DataFrame) dataframe with new column - 'itm_cnt_nxt_mnth', which is sales next month for an item in a shop
    '''
    data_monthly_ext['itm_cnt_nxt_mnth'] = data_monthly_ext.groupby(['shop_id', 'item_id'])['item_cnt_month'].shift(-1)
    return data_monthly_ext

def split_train_test_predict(data_monthly_ext):
    '''
        split_train_test_predict() - function that splits data into train, test and predict, where predict - is the last available month in the data,
                       and for which we will be generating prediction for the next, unknown to us month. Test will be used to see how good is the model.
                       Since we are dealing with timeseries, we cannot simply randomly split data. We also want to split exactly by month periods, so the standard 
                       split function will not work.
        Input:
            data_monthly - (pd.DataFrame) a Pandas dataframe with aggregated sales data
        Output:
            train_set - (pd.DataFrame) training data
            test_set - (pd.DataFrame) testing data to see how we performed
            predict_set - (pd.DataFrame) set of sales for the last month for which we will predict future sales for the next unseen month
    '''
    # calculate date_block_num for splitting
    num_months = data_monthly_ext['date_block_num'].max() + 1
    # starting from the 3rd month as we have rolling statistics with 3 month window (date_block_num starts with 0)
    train_low = 3 - 1
    # get approximately 70% of the data
    train_high = int(0.7 * num_months) - 1
    # testing data will have all the rest of the month up to a last one, which is used for predictiong as it does not have data on next month sales
    test_high = num_months - 1

    train_set = data_monthly_ext[(data_monthly_ext['date_block_num'] >= train_low) & (data_monthly_ext['date_block_num'] <= train_high )].copy()
    test_set = data_monthly_ext[(data_monthly_ext['date_block_num'] > train_high) & (data_monthly_ext['date_block_num'] < test_high )].copy()
    predict_set = data_monthly_ext[data_monthly_ext['date_block_num'] == test_high].copy()

    # in train and test get rid of the null values in target variable
    train_set = train_set.dropna(subset=['itm_cnt_nxt_mnth'])
    test_set = test_set.dropna(subset=['itm_cnt_nxt_mnth'])

    return train_set, test_set, predict_set



def add_set_features(train, test):
    '''
        add_set_features() - function for more feature engineering done on each train and test sets to avoid data leakage.
                            We will be calculating statistics based on the sales in the next month, so cannot use it on predict set as
                            it does not have information on the future sales. On test set we are setting values calculated based on train data
                            because we want to avoid data leakage and test data "knowing" about sales we want to predict.
        Input:
            train_set - (pd.DataFrame) training data
            test_set - (pd.DataFrame) testing data to see how we performed
        Output:
            train_set - (pd.DataFrame) training data with new features
            test_set - (pd.DataFrame) testing data with new features based on training data without data that can help forecast values on test,
                        only past values from train are added here
    '''
    # each new feature will be a statistics computed by grouping and aggregating next month sales by certain dimensions
    # we will generate that grouped statistics and then merge larger ungrouped train and test dataframes with it
    # computing on train data for both train and test as we don't want to give away any information to test set
    # as the calculations are done on future sale and since we are dealing with timeseries

    def generate_statistics(dataset, group_by_columns, new_column_names, agg_column='itm_cnt_nxt_mnth', agg_function_names=['mean']):
        '''
            generate_statistics() - helper internal function for generating statistics of a dataset using grouping and aggregation
            Input:
                dataset - (pd.DataFrame) data frame to calculate statistics on
                group_by_columns - (list of str) columns to group by
                new_column_names - (list of str) how to name new resulting columns
                agg_column - (str) on which column the calculation will be performed
                agg_function_names - (list of str) which aggregate functions to use on 'agg_column' after grouping

            Output:
                res - (pd.DataFrame) dataframe with 'new_column_names' columns - result of grouping and applying aggreagate functions
        '''
        res = dataset.groupby(group_by_columns).agg({agg_column: agg_function_names})
        res.columns = new_column_names
        res.reset_index(inplace=True)
        return res


    # --- Add mean statistic features to train and test sets.

    # Averages for each item across all shops, months and years
    item_means = generate_statistics(train, ['item_id'], ['item_mean_future'])
    train = pd.merge(train, item_means, on=['item_id'], how='left')
    test = pd.merge(test, item_means, on=['item_id'], how='left')

    # Averages for each item in each shop across all months and years
    shop_item_means = generate_statistics(train, ['shop_id', 'item_id'], ['shop_item_mean_future'])
    train = pd.merge(train, shop_item_means, on=['shop_id', 'item_id'], how='left')
    test = pd.merge(test, shop_item_means, on=['shop_id', 'item_id'], how='left')

    # Averages for each shop across all years, months and items
    shop_means = generate_statistics(train, ['shop_id'], ['shop_mean_future'])
    train = pd.merge(train, shop_means, on=['shop_id'], how='left')
    test = pd.merge(test, shop_means, on=['shop_id'], how='left')

    # Averages for each category across all shops, months and years
    category_means = generate_statistics(train, ['item_category_id'], ['category_mean_future'])
    train = pd.merge(train, category_means, on=['item_category_id'], how='left')
    test = pd.merge(test, category_means, on=['item_category_id'], how='left')

    # Averages for each year across all shops, months and items
    year_means = generate_statistics(train, ['year'], ['year_mean_future'])
    train = pd.merge(train, year_means, on=['year'], how='left')
    test = pd.merge(test, year_means, on=['year'], how='left')

    # Averages for each month across all shops, years and items
    month_means = generate_statistics(train, ['month'], ['month_mean_future'])
    train = pd.merge(train, month_means, on=['month'], how='left')
    test = pd.merge(test, month_means, on=['month'], how='left')    

    return train, test


def split_data_labels(train_set, test_set, predict_set):
    '''
        split_data_labels() - function that splits sets into data (features) and labels to predict, 
            specifically train and test into X_train, Y_train, X_test, Y_test 
            and also creates X_predict for prediction in the same format as X_train and X_test.
            X_predict - is the last available month in the data, and for which we will be generating prediction for the next, unknown to us, month. 
            Test will be used to see how good is the model.
            Since we are dealing with timeseries, we cannot simply randomly split data. We also want to split exactly by month periods, so the standard 
            split function will not work.
        Input:
            train_set - (pd.DataFrame) training data
            test_set - (pd.DataFrame) testing data to see how we performed
            predict_set - (pd.DataFrame) set of sales for the last month for which we will predict future sales for the next unseen month
        Output:
            X_train - (pd.DataFrame) training features
            Y_train - (pd.DataFrame) training labels
            X_test - (pd.DataFrame) testing features
            Y_test - (pd.DataFrame) testing labels
            X_predict - (pd.DataFrame) features for predicting unknown data  
    '''
    # create train and test sets and labels. 
    X_train = train_set.drop(['itm_cnt_nxt_mnth', 'date_block_num'], axis=1)
    Y_train = train_set['itm_cnt_nxt_mnth'].astype(int)
    X_test = test_set.drop(['itm_cnt_nxt_mnth', 'date_block_num'], axis=1)
    Y_test = test_set['itm_cnt_nxt_mnth'].astype(int)
    
    # create X_predict to predct next unseen month
    history = pd.concat([train_set, test_set]).drop_duplicates(subset=['item_id'], keep='last')
    X_predict = pd.merge(predict_set, history, on=['item_id'], how='left', suffixes=['', '_'])
    X_predict.drop('itm_cnt_nxt_mnth', axis=1, inplace=True)
    X_predict = X_predict[X_train.columns]

    return X_train, Y_train, X_test, Y_test, X_predict
    

def return_processed_data(data):
    '''
        return_processed_data() - function that combines all of the ETL steps for ML training and predicting.
                                  Raises exception if datafile is not in needed format or has less than 6 month of data
        Input:
            data - (pd.DataFrame) dataframe with sales data.
                    Needed columns are: date, shop_id, item_id, item_price, item_cnt_day, item_name, item_category_id
        Output:
            X_train - (pd.DataFrame) training features
            Y_train - (pd.DataFrame) training labels
            X_test - (pd.DataFrame) testing features
            Y_test - (pd.DataFrame) testing labels
            X_predict - (pd.DataFrame) data set (features) for predicting unknown data
            extended_predict_set - (pd.DataFrame) data set for predicting data, like X_predict, but with all the original columns (like item_id and etc.) to return to a user later
    '''
    data = check_data_correctnes(data)
    data_monthly = clean_and_aggreagate(data)
    data_monthly_ext = add_empty_rows(data_monthly)
    data_monthly_ext = add_global_features(data_monthly_ext)
    data_monthly_ext = add_labels(data_monthly_ext)
    train_set, test_set, predict_set = split_train_test_predict(data_monthly_ext)
    train_set, test_set = add_set_features(train_set, test_set)
    X_train, Y_train, X_test, Y_test, X_predict = split_data_labels(train_set, test_set, predict_set)


    # --- select features that will be used for training, testing and predicting
    features = ['month_mean_future', 'year_mean_future', 'item_mean_future', 'shop_mean_future', 'shop_item_mean_future', 'category_mean_future',
                'category_mean_past', 'item_mean_past', 
                'item_cnt_month', 'item_cnt_day_mean', 
                'item_cnt_roll_mean', 'item_cnt_roll_std', 
                'item_cnt_shifted1', 'item_cnt_shifted2', 'item_cnt_shifted3', 
                'item_trend', 'month']
    # ---- Select subsets
    # First, save the data we will be predicting before we will select features for modelling from it - 'extended_predict_set'. 
    # This will allow us to return prediction in a clear format to the user
    extended_predict_set = X_predict
    # --- subset other Xs            
    X_train = X_train[features]
    X_test = X_test[features]
    X_predict = X_predict[features]

    return X_train, Y_train, X_test, Y_test, X_predict, extended_predict_set

def create_prediction_df(extended_predict_set, Y_predict, columns = ['year', 'month', 'shop_id', 'item_id', 'item_category_id', 'item_price_avg']):
    '''
        create_prediction_df() - a function that combines predicted target data (sales amount) with the data itself (shops, items, categories) into one Pandas dataframe
        Input:
            extended_predict_set - (pd.dataframe) dataframe with data (year, month, shops, items, categories, etc.)
            Y_predict - (list or numpy array) dataframe with predicted sales count for the next month
            columns = columns of extended_predict_set to keep
        Output:
            result_df - resulting df with combined data
    '''    
    result_df = extended_predict_set[columns].copy()

    # get current month and year
    cur_month = extended_predict_set['month'].iloc[0]
    cur_year = extended_predict_set['year'].iloc[0]

    # increase current month by 1 to output prediction
    cur_month += 1
    # check that we are less than 12, set to 1 if 13. '%' alone won't work as there is case of 12 that gives 0
    if cur_month == 13:
        cur_month = 1
        # increae a year by one
        cur_year += 1

    result_df.loc[:,'year'] = cur_year
    result_df.loc[:,'month'] = cur_month

    # add predicted sales
    result_df.loc[:,'predicted_number_items_sold'] =  Y_predict

    return result_df

In [2]:
data = preprocess_data('data/sales_train.csv', 'data/items.csv', use_shop_ids=range(46))

In [3]:
data.shape

(1820364, 7)

In [4]:
data_agg = clean_and_aggreagate(data)

In [5]:
data_agg.head()

,year,month,shop_id,item_category_id,item_id,item_price_avg,item_cnt_month,item_cnt_day_mean
0,2013,1,0,40,32,221.0,6.0,1.5
1,2013,1,0,37,33,347.0,3.0,1.0
2,2013,1,0,40,35,247.0,1.0,1.0
3,2013,1,0,40,43,221.0,1.0,1.0
4,2013,1,0,57,51,128.5,2.0,1.0


In [6]:
data_agg.shape

(984523, 8)

In [7]:
data_agg.describe().T

,count,mean,std,min,25%,50%,75%,max
year,984523.0,2013.787353,0.781577,2013.00,2013.0,2014.0,2014.0,2015.0
month,984523.0,6.101955,3.468553,1.00,3.0,6.0,9.0,12.0
shop_id,984523.0,21.593682,10.097488,0.00,15.0,25.0,29.0,39.0
item_category_id,984523.0,41.519846,16.401264,0.00,30.0,40.0,55.0,83.0
item_id,984523.0,10622.662088,6244.475185,18.00,5033.0,10395.0,16028.0,22169.0
item_price_avg,984523.0,808.740746,1585.617027,0.09,199.0,399.0,899.0,307980.0
item_cnt_month,984523.0,2.328111,9.059973,1.00,1.0,1.0,2.0,2253.0
item_cnt_day_mean,984523.0,1.081716,1.536588,1.00,1.0,1.0,1.0,1000.0


In [8]:
data_agg.tail()

,year,month,shop_id,item_category_id,item_id,item_price_avg,item_cnt_month,item_cnt_day_mean
984518,2015,10,39,83,22088,119.0,5.0,2.500000
984519,2015,10,39,83,22091,179.0,1.0,1.000000
984520,2015,10,39,42,22100,629.0,1.0,1.000000
984521,2015,10,39,42,22102,1250.0,4.0,1.333333
984522,2015,10,39,40,22162,349.0,1.0,1.000000


In [9]:
numb_transactions = data.shape[0]
print('Number of transactions is {}'.format(numb_transactions))

Number of transactions is 1820364


In [10]:
numb_shops = data_agg['shop_id'].nunique()
print('Number of shops is {}'.format(numb_shops))

Number of shops is 40


In [11]:
numb_categories = data_agg['item_category_id'].nunique()
print('Number of categories is {}'.format(numb_categories))

Number of categories is 74


In [12]:
numb_items = data_agg['item_id'].nunique()
print('Number of unique items is {}'.format(numb_items))

Number of unique items is 19111


## 2. Feature engineering  and train, test split

### Steps below will take significant amount of time

In [13]:
X_train, Y_train, X_test, Y_test, X_predict, predict_extended = return_processed_data(data)

KeyboardInterrupt: 

In [ ]:
X_train.describe()

In [ ]:
Y_train.describe()

In [ ]:
X_test.describe()

In [ ]:
Y_test.describe()

In [ ]:
X_predict.describe()

### Long cell of functions from app/train_model.py for training and evaluation, you can scroll down

In [21]:
# import libraries
import sys
import pandas as pd
from sklearn.pipeline import Pipeline
import pickle
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from xgboost import plot_importance

def build_model():
    '''
        build_model() - function that creates a model to later train, test and save for predicting
        Input:
            None 
        Output:
            model - a sklearn GridSearchCV object, model to be trained on existing data and predict categories for the new data
    '''

    
    parameters = {
                #"max_depth"        : [ 1, 2, 6],
                "min_child_weight" : [3, 5]
    }        
    # using GridSearch for optimization is not a good idea for time series since we can only test on future data
    cv_xgb = GridSearchCV(XGBRegressor(), param_grid = parameters, n_jobs=-1, cv=3)
    model = cv_xgb
    #model = XGBRegressor(min_child_weight=10)
    return model


def evaluate_model(model, X_test, y_test):
    '''
        evaluate_model() - function that evaluates an sklearn model
        Input:
            model - a trained sklearn model capable of  'predict' methods
            X_test - data for testing, features
            y_test - labels for testing, targets
        Output:
            MAE - (float) - mean absolute error across all shops and items montly data for period of time defined in X_test dataset
    '''
    # get the model prediction
    y_pred = model.predict(X_test)
    # make integers for meaningfull predictions
    y_pred = y_pred.astype(int)
    MAE = mean_absolute_error(y_test, y_pred)

    validate_act_vs_pred = pd.DataFrame(zip(y_test, y_pred), columns=['actual', 'prediction'])
    validate_non_zero = validate_act_vs_pred[(validate_act_vs_pred['actual'] != 0) | (validate_act_vs_pred['prediction'] != 0)]
    MAE_non_zero = mean_absolute_error(validate_non_zero['actual'], validate_non_zero['prediction'])
    return MAE, MAE_non_zero

## 3. Creating a base estimator to compare with 

## 4. Creating and testing Linear model

In [ ]:
from sklearn.linear_model import LinearRegression
linearModel = LinearRegression()
print('Training model...')
linearModel.fit(X_train, Y_train)
print('Evaluating model...')
mae, mae_non_zero = evaluate_model(linearModel, X_test, Y_test)
print('MAE and MAE non-zero on test data are {} and {}'.format(mae, mae_non_zero) )

## 5. Creating and testing XGBoost model

In [ ]:
print('Building model...')
model = build_model()
        
print('Training model...')
model.fit(X_train, Y_train)
print('XGBRegressor with this parameters turned out to be the best:', model.best_estimator_.get_params())

In [ ]:
print('Evaluating model...')
mae, mae_non_zero = evaluate_model(model, X_test, Y_test)
print('MAE and MAE non-zero on test data are {} and {}'.format(mae, mae_non_zero) )

In [ ]:
print('XGBRegressor with this parameters turned out to be the best:', model.best_estimator_.get_params())

### Saving the model 

In [ ]:
import pickle
print('Saving model...')
file_pkl = open('models/forecast.pkl', 'wb')
pickle.dump(model, file_pkl)
file_pkl.close()

###  See what features are the most important for this model

In [ ]:
import matplotlib.pyplot as plt

plot_importance(model)
plt.show()

## 6. Testing on unseen shops 

In [14]:
data_new = preprocess_data('data/sales_train.csv', 'data/items.csv', use_shop_ids=range(46, 60))

In [15]:
data_new.describe()

,shop_id,item_id,item_price,item_cnt_day,item_category_id
count,123016.000000,123016.000000,123016.000000,123016.000000,123016.000000
mean,46.460875,9634.519518,1005.884712,1.191934,37.876049
std,0.498469,6260.342913,1847.494646,1.406043,17.739795
min,46.000000,28.000000,0.100000,-2.000000,2.000000
25%,46.000000,4231.000000,299.000000,1.000000,23.000000
50%,46.000000,7805.000000,549.000000,1.000000,37.000000
75%,47.000000,15100.000000,1180.000000,1.000000,55.000000
max,47.000000,22167.000000,32990.000000,251.000000,83.000000


In [17]:
X_train_new, Y_train_new, X_test_new, Y_test_new, X_predict_new, predict_extended = return_processed_data(data_new)

In [22]:
import joblib
print('Loading model...')
model = joblib.load("models/forecast_v1.pkl")

print('Evaluating model...')
mae, mae_non_zero = evaluate_model(model, X_test_new, Y_test_new)
print('MAE and MAE non-zero on test data are {} and {}'.format(mae, mae_non_zero) )

Loading model...
[02:53:29] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Evaluating model...
MAE and MAE non-zero on test data are 0.29431850926904235 and 2.8632901929636034


## 7. Conclusions 